In [1]:
import os
import requests
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
from time import sleep

In [2]:
# Spotify API Keys
import config_will as config
os.environ['SPOTIPY_CLIENT_ID']=config.spotify_cliend_id
os.environ['SPOTIPY_CLIENT_SECRET']=config.spotify_client_secret

In [3]:
# Generate API Token
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = client_credentials_manager.get_access_token()
# print(token)

In [4]:
# Load clean combined weekly data
COMBINED_FILE_NAME = os.path.join('data', 'clean', 'half_song_url_list.csv')
combined_wkly_data_df = pd.read_csv(COMBINED_FILE_NAME)

unique_tracks_df = pd.DataFrame({'URL' : combined_wkly_data_df['URL'].unique()})
unique_tracks_df.count()

URL    12622
dtype: int64

In [5]:
features_list = []
for ii, url in enumerate(unique_tracks_df['URL']):
    try:
        track_id = url.split('/')[-1]
        print(f'{url} - {track_id}')
        feature_url = 'https://api.spotify.com/v1/audio-features/{0}'.format(track_id)
        feature_rsp = requests.get(feature_url, headers={'Authorization': 'Bearer '+token})
        features = feature_rsp.json()
    
        track_rsp = requests.get(features['track_href'], headers={'Authorization': 'Bearer '+token})
        track_info = track_rsp.json()
        features_as_dict = {
            'URL':url,
            'Acousticness': features['acousticness'],
            'Danceability': features['danceability'],
            'Duration (ms)': features['duration_ms'],
            'Energy': features['energy'],
            'Instrumentalness': features['instrumentalness'],
            'Key': features['key'],
            'Liveness': features['liveness'],
            'Loudness': features['loudness'],
            'Mode': features['mode'],
            'Speechiness': features['speechiness'],
            'Tempo': features['tempo'],
            'Time Signature': features['time_signature'],
            'Valence': features['valence'],
            'Album': track_info['album']['name'],
            'Image': track_info['album']['images'][1]['url'],
            'Explicit': track_info['explicit'],
            'Popularity': track_info['popularity']
        }
        features_list.append(features_as_dict)
        sleep(3) # Throttle API calls
        
    except:
        print("Error, moving on...")
        token = client_credentials_manager.get_access_token()
        sleep(3)
 
features_df = pd.DataFrame(features_list)
enriched_df = pd.merge(combined_wkly_data_df, features_df, how='left', on='URL')
enriched_df.head()

https://open.spotify.com/track/15VIPlqNYLgN6T43kBAClE - 15VIPlqNYLgN6T43kBAClE
https://open.spotify.com/track/3Jv1XgMPVjdN5xGfnRJ20s - 3Jv1XgMPVjdN5xGfnRJ20s
https://open.spotify.com/track/4d6ihidBTDF0SqwdTDsAyW - 4d6ihidBTDF0SqwdTDsAyW
https://open.spotify.com/track/23NWj2izXAJ4yL6Nah73wf - 23NWj2izXAJ4yL6Nah73wf
https://open.spotify.com/track/2vVaKhH1Pah7Vyp0bJ3HDf - 2vVaKhH1Pah7Vyp0bJ3HDf
https://open.spotify.com/track/03Dpt8Z4Zww4NGJb8503zb - 03Dpt8Z4Zww4NGJb8503zb
https://open.spotify.com/track/2yAVzRiEQooPEJ9SYx11L3 - 2yAVzRiEQooPEJ9SYx11L3
https://open.spotify.com/track/6TGapnqEUkkSBanKyn5qYy - 6TGapnqEUkkSBanKyn5qYy
https://open.spotify.com/track/334S9SWZMjyRDN0wrHw7wk - 334S9SWZMjyRDN0wrHw7wk
https://open.spotify.com/track/2XFP42EK7jg4bGHYXRcxV3 - 2XFP42EK7jg4bGHYXRcxV3
https://open.spotify.com/track/7jWgkIrTDiB3vF8dm5AUyQ - 7jWgkIrTDiB3vF8dm5AUyQ
https://open.spotify.com/track/3yLaeajp3jUl6Xs8MCjxl5 - 3yLaeajp3jUl6Xs8MCjxl5
https://open.spotify.com/track/15BQ7vEDv2LJuh8TxWIht

https://open.spotify.com/track/2xbI8Vmyv3TkpTdywpPyNw - 2xbI8Vmyv3TkpTdywpPyNw
https://open.spotify.com/track/1AH60dMaEfNjzDoUIFwIQi - 1AH60dMaEfNjzDoUIFwIQi
https://open.spotify.com/track/4PzovBqgnSHKd8opsP7IVM - 4PzovBqgnSHKd8opsP7IVM
https://open.spotify.com/track/7oOOI85fVQvVnK5ynNMdW7 - 7oOOI85fVQvVnK5ynNMdW7
https://open.spotify.com/track/77GcYr9JP6uvJM0kPa6Nzk - 77GcYr9JP6uvJM0kPa6Nzk
https://open.spotify.com/track/74u8cWJGP8d8xsdD0pFoWg - 74u8cWJGP8d8xsdD0pFoWg
https://open.spotify.com/track/1eLSF6HfrRA0AsNmTkUlKx - 1eLSF6HfrRA0AsNmTkUlKx
https://open.spotify.com/track/6zaywrggIpaM7Ghgvgjb8R - 6zaywrggIpaM7Ghgvgjb8R
https://open.spotify.com/track/5EOoMWIB9iK4ZpcSex9Ec7 - 5EOoMWIB9iK4ZpcSex9Ec7
https://open.spotify.com/track/5rX6C5QVvvZB7XckETNych - 5rX6C5QVvvZB7XckETNych
https://open.spotify.com/track/1ghSpHC4nGlqWzbPJiVeDU - 1ghSpHC4nGlqWzbPJiVeDU
https://open.spotify.com/track/0IkKz2J93C94Ei4BvDop7P - 0IkKz2J93C94Ei4BvDop7P
https://open.spotify.com/track/0U6DEXxx5D727xgg76jYk

https://open.spotify.com/track/5clFSlfkCRlhnH1cAQjSBi - 5clFSlfkCRlhnH1cAQjSBi
https://open.spotify.com/track/5A2fkyeIbRqwXGkIdadL5q - 5A2fkyeIbRqwXGkIdadL5q
https://open.spotify.com/track/0L0s5w1Oy3Yfr0vKQrltXq - 0L0s5w1Oy3Yfr0vKQrltXq
https://open.spotify.com/track/3t1Eo730ZEHpGJ57J0ME4D - 3t1Eo730ZEHpGJ57J0ME4D
https://open.spotify.com/track/0uJrYOhFXe4Y86JXPPMqt1 - 0uJrYOhFXe4Y86JXPPMqt1
https://open.spotify.com/track/50K0GKbudptd9wZwQlQWdj - 50K0GKbudptd9wZwQlQWdj
https://open.spotify.com/track/2LArYy8hZB9FUbUZvZg8es - 2LArYy8hZB9FUbUZvZg8es
https://open.spotify.com/track/02Wxt6S5N3socdwU5nMvuH - 02Wxt6S5N3socdwU5nMvuH
https://open.spotify.com/track/5C49PNKuD2UtwVl6AAhSZa - 5C49PNKuD2UtwVl6AAhSZa
https://open.spotify.com/track/6GrPaW2OIHuweYcVRoYbmv - 6GrPaW2OIHuweYcVRoYbmv
https://open.spotify.com/track/59nD3VnYWcP43nUNDwWRmD - 59nD3VnYWcP43nUNDwWRmD
https://open.spotify.com/track/475r4PNg7pjWJU981ZCXYs - 475r4PNg7pjWJU981ZCXYs
https://open.spotify.com/track/7oYouEGAQJpU75CAWFEMi

https://open.spotify.com/track/33XMbRw6KmXEFOPGtT8mkO - 33XMbRw6KmXEFOPGtT8mkO
https://open.spotify.com/track/5tRc0MzBeU9xI9Den2jqSU - 5tRc0MzBeU9xI9Den2jqSU
https://open.spotify.com/track/50ldVhQzNcq8d29kiAYWbh - 50ldVhQzNcq8d29kiAYWbh
https://open.spotify.com/track/6ZY14oBfrMJZKpYqLDilwu - 6ZY14oBfrMJZKpYqLDilwu
https://open.spotify.com/track/7hQ3x7TrRK45OFOYqq9HGU - 7hQ3x7TrRK45OFOYqq9HGU
https://open.spotify.com/track/79270nUL3GMDYJylJKj6o7 - 79270nUL3GMDYJylJKj6o7
https://open.spotify.com/track/2ClxcnvQ80QXf7zr23TMA8 - 2ClxcnvQ80QXf7zr23TMA8
https://open.spotify.com/track/2JlXCEpV790w88GVM5soZA - 2JlXCEpV790w88GVM5soZA
https://open.spotify.com/track/5oqOm8uEvF8VNeLpaIHdQX - 5oqOm8uEvF8VNeLpaIHdQX
https://open.spotify.com/track/6JlJmYiOf1HyuVOztQCRW1 - 6JlJmYiOf1HyuVOztQCRW1
https://open.spotify.com/track/1dQMu08D8bQPYrPGSYl1zF - 1dQMu08D8bQPYrPGSYl1zF
https://open.spotify.com/track/64P2kTsP4MOhjV2DNbXKEW - 64P2kTsP4MOhjV2DNbXKEW
https://open.spotify.com/track/1zx1U7z4LwsGDlPnUDwzV

https://open.spotify.com/track/4PI1kf6n54YFT60EfFbyYK - 4PI1kf6n54YFT60EfFbyYK
https://open.spotify.com/track/3rJNOxdH0xlEueLgDGylQ4 - 3rJNOxdH0xlEueLgDGylQ4
https://open.spotify.com/track/3eoW5TQr9MjwpOu2y8HAV3 - 3eoW5TQr9MjwpOu2y8HAV3
https://open.spotify.com/track/5QE9EfDAQKOUMzcRY0vuFC - 5QE9EfDAQKOUMzcRY0vuFC
https://open.spotify.com/track/5hP5QSJ93AoDxsODWBMtMu - 5hP5QSJ93AoDxsODWBMtMu
https://open.spotify.com/track/73geHIgVIIwZyyjprCRb81 - 73geHIgVIIwZyyjprCRb81
https://open.spotify.com/track/7uEspqmUl5WiO30SeBA2fr - 7uEspqmUl5WiO30SeBA2fr
https://open.spotify.com/track/1DxTCJAqXjdRw3uK0Oul48 - 1DxTCJAqXjdRw3uK0Oul48
https://open.spotify.com/track/3STVVZnLg3XSMNmAucvosa - 3STVVZnLg3XSMNmAucvosa
https://open.spotify.com/track/5AHevkKIxwliPEeTcOepiG - 5AHevkKIxwliPEeTcOepiG
https://open.spotify.com/track/2URhqIUd69hFpVN5aj400p - 2URhqIUd69hFpVN5aj400p
https://open.spotify.com/track/7i99ReSqfeczptxhuoGzPQ - 7i99ReSqfeczptxhuoGzPQ
https://open.spotify.com/track/5YF4MMrZ1Y0t4vhaU3VgM

...

https://open.spotify.com/track/414Oq8c5PkwZYVSHhjFn2k - 414Oq8c5PkwZYVSHhjFn2k
https://open.spotify.com/track/5rPkcdE6zEDVygNJHyRSSc - 5rPkcdE6zEDVygNJHyRSSc
https://open.spotify.com/track/2qcpQ3qNI8OHZuyafsqC0l - 2qcpQ3qNI8OHZuyafsqC0l
https://open.spotify.com/track/1qyWnszJ8f9cMx5ufcJ3nN - 1qyWnszJ8f9cMx5ufcJ3nN
https://open.spotify.com/track/2DEMSm4IWrmsBupLRglxRn - 2DEMSm4IWrmsBupLRglxRn
https://open.spotify.com/track/78zL2RIYaIyV7wT65ntOrg - 78zL2RIYaIyV7wT65ntOrg
https://open.spotify.com/track/7o2ADG8zFvdz8Rs4aLslcA - 7o2ADG8zFvdz8Rs4aLslcA
https://open.spotify.com/track/1FGioiPySuGaCqR09nEYDs - 1FGioiPySuGaCqR09nEYDs
https://open.spotify.com/track/2yNpNdewag1fQduIJUKT8n - 2yNpNdewag1fQduIJUKT8n
https://open.spotify.com/track/5xieAAW2HASJRIqZya0bwc - 5xieAAW2HASJRIqZya0bwc
https://open.spotify.com/track/1EonLMgCtLpzkUCuk5DuCs - 1EonLMgCtLpzkUCuk5DuCs
https://open.spotify.com/track/28nC8fLwLlgtJrCFDGcqQW - 28nC8fLwLlgtJrCFDGcqQW
https://open.spotify.com/track/0AzBtsy4sR7lTY6WydHDB

https://open.spotify.com/track/3uWiCjrrWDVr6VjvDajWnT - 3uWiCjrrWDVr6VjvDajWnT
https://open.spotify.com/track/1kN6sZ5YxxAnZXSQvzQPcx - 1kN6sZ5YxxAnZXSQvzQPcx
https://open.spotify.com/track/3xlDQjuiNPAI3CT79CKBbZ - 3xlDQjuiNPAI3CT79CKBbZ
https://open.spotify.com/track/2cxbp71Cw0RvPi8bY2uf8q - 2cxbp71Cw0RvPi8bY2uf8q
https://open.spotify.com/track/57nGlZyEM568t0hwbh1hzT - 57nGlZyEM568t0hwbh1hzT
https://open.spotify.com/track/3FFNVtDMJgE83qnOfuCmZA - 3FFNVtDMJgE83qnOfuCmZA
https://open.spotify.com/track/7F7WJafflMnZtTQNm3UMtJ - 7F7WJafflMnZtTQNm3UMtJ
https://open.spotify.com/track/2s1WH3X0wf0EuQKsKEtacK - 2s1WH3X0wf0EuQKsKEtacK
https://open.spotify.com/track/42NRi1QfEp3JBpcUXUhgYX - 42NRi1QfEp3JBpcUXUhgYX
https://open.spotify.com/track/0vjgfBtp5g85bzRYIbbsGU - 0vjgfBtp5g85bzRYIbbsGU
https://open.spotify.com/track/53j756p2sMyhQsSkQQuxl6 - 53j756p2sMyhQsSkQQuxl6
https://open.spotify.com/track/7oeLuqjxrTEcWJF0spqPZM - 7oeLuqjxrTEcWJF0spqPZM
https://open.spotify.com/track/1rdEvKn4U7ylrJD3l5hNe

https://open.spotify.com/track/5ITA4zYXlMNP8e0bPnDAR4 - 5ITA4zYXlMNP8e0bPnDAR4
https://open.spotify.com/track/1Tqno4i730eOD1RUlyZU9Z - 1Tqno4i730eOD1RUlyZU9Z
https://open.spotify.com/track/3PDPQQcv0dIVy6iau6txti - 3PDPQQcv0dIVy6iau6txti
https://open.spotify.com/track/0gaoAjyOeLSQAHuN4rXwYP - 0gaoAjyOeLSQAHuN4rXwYP
https://open.spotify.com/track/76yE1RKhbsl7BJwX2gwOPX - 76yE1RKhbsl7BJwX2gwOPX
https://open.spotify.com/track/1fSBadVQEPMQ25HvLYNqBu - 1fSBadVQEPMQ25HvLYNqBu
https://open.spotify.com/track/21euJbPr2LZaEH10U9rLQ8 - 21euJbPr2LZaEH10U9rLQ8
https://open.spotify.com/track/0uBeWkmQPnPif9zg7M8Wbc - 0uBeWkmQPnPif9zg7M8Wbc
https://open.spotify.com/track/2DidCf9OOqDqHl1PJbPFSk - 2DidCf9OOqDqHl1PJbPFSk
https://open.spotify.com/track/35GWsHQ58wmBmQGLxkEZ0I - 35GWsHQ58wmBmQGLxkEZ0I
https://open.spotify.com/track/0jg8mUxnDWdoN5IlDdR9DP - 0jg8mUxnDWdoN5IlDdR9DP
https://open.spotify.com/track/3FX1AO8gTEBSJVqRCmeBfK - 3FX1AO8gTEBSJVqRCmeBfK
https://open.spotify.com/track/3btVFJg2I094SKox8vCN7

https://open.spotify.com/track/13jmwQ59S7oSkTF5l8UE6Z - 13jmwQ59S7oSkTF5l8UE6Z
https://open.spotify.com/track/5rg9SNuAv2G7jXAaZvUwWx - 5rg9SNuAv2G7jXAaZvUwWx
https://open.spotify.com/track/0t9kFan50ByqfwFGtV0ofA - 0t9kFan50ByqfwFGtV0ofA
https://open.spotify.com/track/3d8IXXKCpdmc9By3yHuqoT - 3d8IXXKCpdmc9By3yHuqoT
https://open.spotify.com/track/39oxt5wWnddSWNM77xfc5i - 39oxt5wWnddSWNM77xfc5i
https://open.spotify.com/track/1J4hDoqoRkeHMNiDfnVw5I - 1J4hDoqoRkeHMNiDfnVw5I
https://open.spotify.com/track/5rsvjbXFPBru3Fw59jznLx - 5rsvjbXFPBru3Fw59jznLx
https://open.spotify.com/track/30WAOSzH0shtlLMjY2x6br - 30WAOSzH0shtlLMjY2x6br
https://open.spotify.com/track/7Ek2FFpR6LAFmuZthNiaU4 - 7Ek2FFpR6LAFmuZthNiaU4
https://open.spotify.com/track/6H5077VTkz7amT9vDH6e64 - 6H5077VTkz7amT9vDH6e64
https://open.spotify.com/track/59xSED9dc86bYL39V93GeV - 59xSED9dc86bYL39V93GeV
https://open.spotify.com/track/1t2HMMHB30mzDs97mk4Fjk - 1t2HMMHB30mzDs97mk4Fjk
https://open.spotify.com/track/6bppK721BKGPEWzTXqvHb

,URL,Acousticness,Album,Danceability,Duration (ms),Energy,Explicit,Image,Instrumentalness,Key,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,Time Signature,Valence
0,https://open.spotify.com/track/15VIPlqNYLgN6T4...,0.1840,pop a pill,0.688,180808.0,0.551,True,https://i.scdn.co/image/8f23487561c28f220873b5...,0.000000,0.0,0.1590,-6.299,0.0,67.0,0.2290,162.118,4.0,0.5580
1,https://open.spotify.com/track/3Jv1XgMPVjdN5xG...,0.0684,Excuse My French,0.734,303533.0,0.893,True,https://i.scdn.co/image/64117d4092c32d69a3078e...,0.000000,2.0,0.7190,-1.847,1.0,60.0,0.1250,138.050,4.0,0.5950
2,https://open.spotify.com/track/4d6ihidBTDF0Sqw...,0.1340,Party Pack EP,0.836,215107.0,0.489,True,https://i.scdn.co/image/b9eb3e80704d6d6754dbfe...,0.000000,0.0,0.0805,-10.484,1.0,59.0,0.3060,147.055,4.0,0.7320
3,https://open.spotify.com/track/23NWj2izXAJ4yL6...,0.2340,Pop Style,0.795,209453.0,0.482,True,https://i.scdn.co/image/e312b96e13cda45938b8ea...,0.000004,11.0,0.1040,-8.995,1.0,58.0,0.2200,132.985,4.0,0.0602
4,https://open.spotify.com/track/2vVaKhH1Pah7Vyp...,0.2190,My Pop Christmas,0.670,254253.0,0.806,False,https://i.scdn.co/image/0b629cc34833b5a3e891b7...,0.000000,0.0,0.2220,-4.823,1.0,63.0,0.0263,147.073,4.0,0.6890


In [6]:
ENRICHED_FILE_NAME = os.path.join('data', 'clean', 'all_genres_song_list_enriched.csv')

if not os.path.exists(os.path.dirname(ENRICHED_FILE_NAME)):
    os.mkdir(os.path.dirname(ENRICHED_FILE_NAME))
    
enriched_df.to_csv(ENRICHED_FILE_NAME, index=False)

In [7]:
# Test Track ID (replace end of URL with Track ID)
feature_url = 'https://api.spotify.com/v1/audio-features/6bzceiEr9C3UlK13ucWfhZ'.format(track_id)
feature_rsp = requests.get(feature_url, headers={'Authorization': 'Bearer '+token})
features = feature_rsp.json()
print(features)

{'danceability': 0.747, 'energy': 0.521, 'key': 9, 'loudness': -10.636, 'mode': 0, 'speechiness': 0.217, 'acousticness': 0.603, 'instrumentalness': 0, 'liveness': 0.257, 'valence': 0.42, 'tempo': 139.991, 'type': 'audio_features', 'id': '6bzceiEr9C3UlK13ucWfhZ', 'uri': 'spotify:track:6bzceiEr9C3UlK13ucWfhZ', 'track_href': 'https://api.spotify.com/v1/tracks/6bzceiEr9C3UlK13ucWfhZ', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6bzceiEr9C3UlK13ucWfhZ', 'duration_ms': 156493, 'time_signature': 4}


In [8]:
# print(client_credentials_manager.is_token_expired(token))
# token = client_credentials_manager.get_access_token()
print(token)

BQDfgnX-kQRmMxv9cP4j9O05bXRxOtgbQoJtOXVfrp-iDoBPXeH_QlKlU87p82vheadLudIOlrhtcw03hFc
